In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import gymnasium as gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions.categorical import Categorical
import matplotlib.pyplot as plt

In [3]:
import sys
sys.argv = [""]

In [4]:
from cart_pole.utils import *
from cart_pole.agent import *
from torch.utils.tensorboard import SummaryWriter
from cart_pole.environment import *
import pdb

In [5]:
args = parse_args(seconds_since_epoch=1764177877)
writer = init_tracking(args)

In [74]:
envs = make_sync_vector_env(args)
agent = Agent(envs).to(args.device)
storage = Storage(args, envs)
optimizer = optim.Adam(agent.parameters(),
                       lr=args.learning_rate,
                       eps=1e-5)

num_updates = args.total_timesteps // args.batch_size
storage.rollout(agent, envs)

In [62]:
import copy


In [163]:
class MinMaxScaler(object):

    def __init__(self,
                 args,
                 envs):

        obs_space = copy.deepcopy(envs.single_observation_space)

        self.is_bounded =\
            np.logical_and(obs_space.bounded_above,
                           obs_space.bounded_below)

        self.is_unbounded =\
            np.logical_not(self.is_bounded)

        self.is_bounded = torch.tensor(self.is_bounded)
        self.is_unbounded = torch.tensor(self.is_unbounded)
        
        self.low = torch.tensor(obs_space.low[self.is_bounded])
        
        self.inv_range =\
            1 / (torch.tensor(obs_space.high[self.is_bounded]) - self.low)

        self.inv_range = self.inv_range.to(args.device)
        self.low = self.low.to(args.device)

    def __call__(self,
                 x):

        scaled_x = x.clone()
        bounds_shape = [1] * (x.ndim - 1) + [self.low.shape[0]]

        is_bounded_nd = torch.zeros(scaled_x.shape,
                                    dtype=torch.bool)

        is_bounded_nd.masked_fill(self.is_bounded, True)

        is_unbounded_nd = torch.zeros(scaled_x.shape,
                                      dtype=torch.bool)

        is_bounded_nd.masked_fill(self.is_bounded, True)
        
obs_scaler = MinMaxScaler(args, envs)
#obs_scaler(storage.obs[0,:,:]) 

In [167]:
x = storage.obs.clone()


tensor([[[ True, False,  True, False],
         [ True, False,  True, False],
         [ True, False,  True, False],
         [ True, False,  True, False]],

        [[ True, False,  True, False],
         [ True, False,  True, False],
         [ True, False,  True, False],
         [ True, False,  True, False]],

        [[ True, False,  True, False],
         [ True, False,  True, False],
         [ True, False,  True, False],
         [ True, False,  True, False]],

        ...,

        [[ True, False,  True, False],
         [ True, False,  True, False],
         [ True, False,  True, False],
         [ True, False,  True, False]],

        [[ True, False,  True, False],
         [ True, False,  True, False],
         [ True, False,  True, False],
         [ True, False,  True, False]],

        [[ True, False,  True, False],
         [ True, False,  True, False],
         [ True, False,  True, False],
         [ True, False,  True, False]]])